In [ ]:
import IPython.display as ipd
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt
import sounddevice as sd
from sdr.audio import Stream
from pathlib import Path
import subprocess
import time

import numpy as np

In [ ]:
launch_wsl = subprocess.Popen(["wsl", "-d", "Ubuntu"])
subprocess.call(["usbipd", "attach", "-b", "1-14", "-w"])

In [ ]:
def yield_frames(command, blocksize, t=5):
    bytes_per_frame = 2  # 16-bit integer
    read_length = blocksize * bytes_per_frame
    
    # Start the subprocess with the provided command
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    
    # Read binary data from the subprocess stdout in chunks
    t0 = time.time()
    while (time.time() - t0) < t:
        # Read a chunk of binary data (16 bits = 2 bytes)
        chunk = process.stdout.read(read_length)
        
        # If no more data is available, break the loop
        if not chunk:
            break
        
        # Convert the binary data to an integer (assuming little-endian byte order)
        block = np.frombuffer(chunk, dtype="int16").reshape((blocksize, 1))
        
        # Yield the integer data
        yield block

    # Close the subprocess after reading all data
    process.stdout.close()
    process.wait()

In [ ]:
blocksize=2048
rate = 44100
frequency_mhz = 94.9
cmd = [
    "wsl", "--exec", "rtl_fm", "-f", f"{frequency_mhz}M", "-s", str(rate*5), "-r", str(rate)
]

In [ ]:
s = Stream(
    source=yield_frames(
        cmd, 
        blocksize,
        t=30
    ), 
    rate=rate,
    blocksize=blocksize,
    channels=1
)
s.start()